In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
def multitime_to_pd(list):
    df = pd.DataFrame(columns = ['Mean', 'Std.Dev.', 'Min', 'Median', 'Max'], index = ['real', 'user', 'sys'])
    for i in range (1,4):
        row = re.sub(' +', ' ', list[i]).split(" ")[:6]
        df.loc[row[0]] = row[1:]        
    return df


def plot_times(df):
    ax = df.plot(x='name', y='time', kind='bar')
    ax.set_xlabel(None)
    ax.set_ylabel('Execution time (s)')
    ax.set_title('Execution Time Comparison')
    
    for i, v in enumerate(df['time']):
        ax.text(i, v, str(v), ha='center', va='bottom', rotation=90)
        
    y_max = ax.get_ylim()
    ax.set_ylim(0, y_max[1] + y_max[1]*0.3) #add 30% of top padding
    plt.show()
    
    
def get_c_times():
    dfO0=multitime_to_pd(open("./compiled/C/O0/c.time", "r").readlines())
    dfO1=multitime_to_pd(open("./compiled/C/O1/c.time", "r").readlines())
    dfO2=multitime_to_pd(open("./compiled/C/O2/c.time", "r").readlines())
    dfO3=multitime_to_pd(open("./compiled/C/O3/c.time", "r").readlines())
    dfOfast=multitime_to_pd(open("./compiled/C/Ofast/c.time", "r").readlines())
    dfOs=multitime_to_pd(open("./compiled/C/Os/c.time", "r").readlines())
    
    name_values = ['O0', 'O1', 'O2', 'O3', 'Ofast', 'Os']
    
    time_values = [float(dfO0.loc['real', 'Mean']), float(dfO1.loc['real', 'Mean']), float(dfO2.loc['real', 'Mean']), 
                   float(dfO3.loc['real', 'Mean']), float(dfOfast.loc['real', 'Mean']), float(dfOs.loc['real', 'Mean'])]
    
    size_values = [os.stat("./compiled/C/O0/benchmark").st_size, os.stat("./compiled/C/O1/benchmark").st_size, os.stat("./compiled/C/O3/benchmark").st_size, 
                   os.stat("./compiled/C/O3/benchmark").st_size, os.stat("./compiled/C/Ofast/benchmark").st_size, os.stat("./compiled/C/Os/benchmark").st_size]
    
    return pd.DataFrame({'opt. flag': name_values, 'size (B)': size_values, 'time (s)': time_values})


def get_clang_times():
    
    compiler="clang"
    name_values = ['O0', 'O1', 'O2', 'O3', 'Ofast', 'Oz']
    size_values = [os.stat("./compiled/WASM/O0/"+compiler+".wasm").st_size, os.stat("./compiled/WASM/O1/"+compiler+".wasm").st_size, 
                   os.stat("./compiled/WASM/O3/"+compiler+".wasm").st_size, os.stat("./compiled/WASM/O3/"+compiler+".wasm").st_size, 
                   os.stat("./compiled/WASM/Ofast/"+compiler+".wasm").st_size, os.stat("./compiled/WASM/Oz/"+compiler+".wasm").st_size]
    
    df = pd.DataFrame({'opt. flag': name_values, 'size (B)': size_values})
    
    for runtime in ["wasmtime", "wasmedge", "wamr"]:
        dfO0=multitime_to_pd(open("./compiled/WASM/O0/"+compiler+"-"+runtime+".time", "r").readlines())
        dfO1=multitime_to_pd(open("./compiled/WASM/O1/"+compiler+"-"+runtime+".time", "r").readlines())
        dfO2=multitime_to_pd(open("./compiled/WASM/O2/"+compiler+"-"+runtime+".time", "r").readlines())
        dfO3=multitime_to_pd(open("./compiled/WASM/O3/"+compiler+"-"+runtime+".time", "r").readlines())
        dfOfast=multitime_to_pd(open("./compiled/WASM/Ofast/"+compiler+"-"+runtime+".time", "r").readlines())
        dfOs=multitime_to_pd(open("./compiled/WASM/Oz/"+compiler+"-"+runtime+".time", "r").readlines())
    
        df[runtime+" (s)"] = [float(dfO0.loc['real', 'Mean']), float(dfO1.loc['real', 'Mean']), float(dfO2.loc['real', 'Mean']), 
                       float(dfO3.loc['real', 'Mean']), float(dfOfast.loc['real', 'Mean']), float(dfOs.loc['real', 'Mean'])]
    
    return df


def get_emcc_times():
    
    compiler="emcc"
    name_values = ['O0', 'O1', 'O2', 'O3', 'Oz']
    size_values = [os.stat("./compiled/WASM/O0/"+compiler+".wasm").st_size, os.stat("./compiled/WASM/O1/"+compiler+".wasm").st_size, 
                   os.stat("./compiled/WASM/O3/"+compiler+".wasm").st_size, os.stat("./compiled/WASM/O3/"+compiler+".wasm").st_size, 
                   os.stat("./compiled/WASM/Oz/"+compiler+".wasm").st_size]
    
    df = pd.DataFrame({'opt. flag': name_values, 'size (B)': size_values})
    
    for runtime in ["wasmtime", "wasmedge", "wamr"]:
        dfO0=multitime_to_pd(open("./compiled/WASM/O0/"+compiler+"-"+runtime+".time", "r").readlines())
        dfO1=multitime_to_pd(open("./compiled/WASM/O1/"+compiler+"-"+runtime+".time", "r").readlines())
        dfO2=multitime_to_pd(open("./compiled/WASM/O2/"+compiler+"-"+runtime+".time", "r").readlines())
        dfO3=multitime_to_pd(open("./compiled/WASM/O3/"+compiler+"-"+runtime+".time", "r").readlines())
        dfOs=multitime_to_pd(open("./compiled/WASM/Oz/"+compiler+"-"+runtime+".time", "r").readlines())
    
        df[runtime+" (s)"] = [float(dfO0.loc['real', 'Mean']), float(dfO1.loc['real', 'Mean']), float(dfO2.loc['real', 'Mean']), 
                       float(dfO3.loc['real', 'Mean']),  float(dfOs.loc['real', 'Mean'])]
    
    return df


In [3]:
get_c_times()

,opt. flag,size (B),time (s)
0,O0,17360,0.197
1,O1,17104,0.059
2,O2,17160,0.051
3,O3,17160,0.053
4,Ofast,18664,0.053
5,Os,17104,0.058


In [4]:
get_clang_times()

,opt. flag,size (B),wasmtime (s),wasmedge (s),wamr (s)
0,O0,33056,0.265,61.642,8.493
1,O1,27248,0.093,8.247,2.067
2,O2,25916,0.088,7.015,1.547
3,O3,25916,0.083,7.118,1.547
4,Ofast,25948,0.076,7.335,1.514
5,Oz,25836,0.095,8.102,1.968


In [5]:
get_emcc_times()

,opt. flag,size (B),wasmtime (s),wasmedge (s),wamr (s)
0,O0,25108,0.274,62.646,8.459
1,O1,19219,0.104,9.687,2.300
2,O2,15698,0.127,8.196,1.757
3,O3,15698,0.085,8.171,1.747
4,Oz,15298,0.109,10.031,2.337
